In [1]:
import geemap
import ee
import os
import numpy as np

In [ ]:
#!python -m pip install geemap
#!export PATH="/home/u9999ad7367849478acc6568ce06912b/.local/bin:$PATH"
#!python -m pip install dask[complete]
#!pip install earthengine-api
#!pip install geemap
#!pip install --user dask[dataframe]

In [ ]:
#ee.Authenticate()

# Define Functions to Retrive, Display, and Save Images

In [2]:
def RetrieveNAIPimage(lat, long, deltalr = 0.00165, deltaud = 0.00135, date1 = '2018-01-01', date2 = '2020-12-31'):
    region = ee.Geometry.BBox(long- deltalr, lat - deltaud, long + deltalr, lat + deltaud)
    dataset = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.date(date1, date2) ).filter(ee.Filter.bounds(region))
    return dataset

def DisplayMap(dataset, lat, long, verbose = True, deltalr = 0.00165, deltaud = 0.00135, zoom = 18):  
    image = dataset.first()
    trueColor = dataset.select(['R', 'G', 'B'])
    trueColorVis = {
      min: 50,
      max: 240,
    }
    if verbose:
        display("images found ",dataset.size())
        print(geemap.image_props(image).getInfo())
        print(geemap.image_stats(image, scale=18).getInfo())
        display(image.geometry().getInfo())
    region = ee.Geometry.BBox(long- deltalr, lat - deltaud, long + deltalr, lat + deltaud)
    regionDebug = ee.Geometry.Polygon(image.getInfo()['properties']['system:footprint']['coordinates'])
    Map = geemap.Map(center=(lat, long), zoom=18)
    Map.addLayer(trueColor, trueColorVis, 'True Color');
    Map.addLayer(region, {'color': 'red'}, '500x500 Image Selected')
    Map.addLayer(regionDebug, {'color': 'green'}, 'NAIP Image Boundary')
    #Map.setOptions('SATELLITE');
    image = dataset.first()

    return Map

def SaveNAIPimage(image ,lat, long, folder, filename, verbose = True, deltalr = 0.00165, deltaud = 0.00135):
    from PIL import Image
    import numpy as np
    fileError.append((folder,filename, lat, long))
    fileTried.append((folder,filename, lat, long, False))
    try:
        region = ee.Geometry.BBox(long - deltalr, lat - deltaud, long + deltalr, lat + deltaud)
        rgb_img = geemap.ee_to_numpy(image, bands=['R', 'G', 'B'], region=region)

#         rgb_img[:,2] = rgb_img[:,2] 
#         rgb_img[:,1] = rgb_img[:,1] 
       
        if verbose:
            print(rgb_img.shape)
            print(image.getInfo()['properties']['system:footprint']['coordinates'])
        PIL_image = Image.fromarray(np.uint8(rgb_img)).convert('RGB')
        # Setting the points for cropped image
        left = 9
        top = 0 
        right = 500
        bottom = 500
         
        # Cropped image of above dimension
        # (It will not change original image)
        PIL_image = PIL_image.crop((left, top, right, bottom))
             
        PIL_image.save(f"{folder}/{filename}.png")
        fileError.remove((folder,filename, lat, long))
        fileTried.remove((folder,filename, lat, long, False))
        fileTried.append((folder,filename, lat, long, True))
    except:
        print(f'{(folder,filename, lat, long)}\nThere was a problem:\nLikely the requested image lies outside or overlaps two NAIP images\nOR deltalr and deltaud may be too large for this coordinate - try reducing - the request image is too large' )

In [3]:
# from PIL import Image 
# im = Image.open("Downloads/Morrison/151swe1.png")
# width, height = im.size
# print(width, height)
# # Setting the points for cropped image
# left = 9
# top = 0 
# right = width
# bottom = 500
 
# # Cropped image of above dimension
# # (It will not change original image)
# im1 = im.crop((left, top, right, bottom))
# im1

# Display Map

Red region is area to be saved

In [5]:
import geemap
import ee
import os
# If it is not yet authenticated, uncomment the line below
#ee.Authenticate()

ee.Initialize()

long, lat = -106.922, 35.5123  # MM good
long, lat = -109.30146919370144, 40.44057649432536 # DNM


dataset = RetrieveNAIPimage(lat, long)
Map = DisplayMap(dataset, lat, long, verbose = True, zoom = 18)
Map

Enter verification code:  4/1AfJohXlzs1qVVECWpMSN7RtCFMOjriVL1nfxytf8lXUI6cA5pG1UT3MZpvE



Successfully saved authorization token.


'images found '

{'IMAGE_DATE': '2018-09-10', 'NOMINAL_SCALE': 0.6, 'system:asset_size': '492.969715 MB', 'system:band_names': ['R', 'G', 'B', 'N'], 'system:id': 'USDA/NAIP/DOQQ/m_4010938_ne_12_060_20180910', 'system:index': 'm_4010938_ne_12_060_20180910', 'system:time_end': '2018-09-10 00:00:00', 'system:time_start': '2018-09-10 00:00:00', 'system:version': 1584779809150912}
{'max': {'B': 203, 'G': 206, 'N': 210, 'R': 207}, 'mean': {'B': 102.3803745795375, 'G': 121.16032230906237, 'N': 175.5390434501865, 'R': 128.88745358421403}, 'min': {'B': 35, 'G': 31, 'N': 31, 'R': 26}, 'std': {'B': 28.071456177398634, 'G': 30.819411461475234, 'N': 23.146170546525116, 'R': 33.46353217341902}, 'sum': {'B': 13313702.098039199, 'G': 15755875.517647048, 'N': 22827368.43529431, 'R': 16760723.607843162}}


{'type': 'Polygon',
 'coordinates': [[[-109.24831419354732, 40.43517348919539],
   [-109.24831410280427, 40.435174179219196],
   [-109.24659370860434, 40.501338191977624],
   [-109.2465965216283, 40.5013407008658],
   [-109.24659888093875, 40.50134351253056],
   [-109.24660006059396, 40.50134368336769],
   [-109.3142308612582, 40.50235027449777],
   [-109.3142342192214, 40.50234812192214],
   [-109.31423784945191, 40.50234629635841],
   [-109.31423803096345, 40.502345666323585],
   [-109.31423812171921, 40.5023449771696],
   [-109.3158922391268, 40.43617860998536],
   [-109.31588942568838, 40.4361761003902],
   [-109.31588697527431, 40.43617330324803],
   [-109.3158858862014, 40.436173117142665],
   [-109.24832136224792, 40.43516888154977],
   [-109.2483180047552, 40.435171033870105],
   [-109.2483143750334, 40.43517278352931],
   [-109.24831419354732, 40.43517348919539]]]}

Map(center=[40.44057649432536, -109.30146919370144], controls=(WidgetControl(options=['position', 'transparent…

# Find name of larger maps from NAIP

Yrun Verbosity to True to see more iamge stats:
- near Dinosaur National Monument we find the NAIP image depicted by the green boundary in the map above.
- m_4010938_ne_12_060_20180910. 

I use USGS Earth Explorer to easily grab that map

# Images from NAIP: Public Domain

https://data.nal.usda.gov/dataset/naip-digital-ortho-photo-image-geospatialdatapresentationform-remote-sensing-image

In [6]:
dataset.getInfo()['properties']['description']

'<p>The National Agriculture Imagery Program (NAIP) acquires aerial imagery\nduring the agricultural growing seasons in the continental U.S.</p><p>NAIP projects are contracted each year based upon available funding and the\nimagery acquisition cycle. Beginning in 2003, NAIP was acquired on\na 5-year cycle. 2008 was a transition year, and a three-year cycle began\nin 2009.</p><p>NAIP imagery is acquired at a one-meter ground sample distance (GSD) with a\nhorizontal accuracy that matches within six meters of photo-identifiable\nground control points, which are used during image inspection.</p><p>Older images were collected using 3 bands (Red, Green, and Blue: RGB), but\nnewer imagery is usually collected with an additional near-infrared band\n(RGBN). RGB asset ids begin with &#39;n<em>&#39;, NRG asset ids begin with &#39;c</em>&#39;, RGBN\nasset ids begin with &#39;m_&#39;.</p><p><b>Provider: <a href="https://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-programs/naip

In [7]:
# Debug
from PIL import Image
import numpy as np

dataset = RetrieveNAIPimage(lat, long)
image = dataset.first()
# folder = 'Downloads/NotMorrison'
# filename = 'bob'
# verbose = False
deltalr = 0.00164
deltaud = 0.00134
region = ee.Geometry.BBox(long - deltalr, lat - deltaud, long + deltalr, lat + deltaud)
rgb_img = geemap.ee_to_numpy(image, bands=['R', 'G', 'B'], region=region)

# region = ee.Geometry.BBox(long - deltalr, lat - deltaud, long + deltalr, lat + deltaud)
# rgb_img = geemap.ee_to_numpy(image, bands=['R', 'G', 'B'], region=region)

# rgb_img[:,2] = rgb_img[:,2] 
# rgb_img[:,1] = rgb_img[:,1] 

# if verbose:
#     print(rgb_img.shape)
#     print(image.getInfo()['properties']['system:footprint']['coordinates'])
# PIL_image = Image.fromarray(np.uint8(rgb_img)).convert('RGB')
# # Setting the points for cropped image
# left = 9
# top = 0 
# right = 500
# bottom = 500
 
# # Cropped image of above dimension
# # (It will not change original image)
# PIL_image = PIL_image.crop((left, top, right, bottom))
     
# PIL_image.save(f"{folder}/{filename}.png")

# Save Image

Must reduce dataset down to a single image via filtering and perhaps median aggregation


In [8]:
# from PIL import Image
# import numpy as np
# fileError = []
# fileTried = []
# dataset = RetrieveNAIPimage(lat, long)
# image = dataset.first()
# folder = 'Downloads/NotMorrison'
# filename = 'bob'
# SaveNAIPimage(image, lat, long, folder, filename, verbose = False)


# Get Coordinates of Region of interest

In [9]:
#ee.Geometry.Polygon(image.getInfo()['properties']['system:footprint']['coordinates'])
image.getInfo()['properties']['system:footprint']['coordinates']

[[-109.24831419354732, 40.43517348919539],
 [-109.24831410280427, 40.435174179219196],
 [-109.24659370860434, 40.501338191977624],
 [-109.2465965216283, 40.5013407008658],
 [-109.24659888093875, 40.50134351253056],
 [-109.24660006059396, 40.50134368336769],
 [-109.3142308612582, 40.50235027449777],
 [-109.3142342192214, 40.50234812192214],
 [-109.31423784945191, 40.50234629635841],
 [-109.31423803096345, 40.502345666323585],
 [-109.31423812171921, 40.5023449771696],
 [-109.3158922391268, 40.43617860998536],
 [-109.31588942568838, 40.4361761003902],
 [-109.31588697527431, 40.43617330324803],
 [-109.3158858862014, 40.436173117142665],
 [-109.24832136224792, 40.43516888154977],
 [-109.2483180047552, 40.435171033870105],
 [-109.2483143750334, 40.43517278352931],
 [-109.24831419354732, 40.43517348919539]]

In [10]:
import pandas as pd
import time
time.time()
SampleTheseCoords = pd.read_csv('SampleTheseCoordsEdited.csv')
arr = SampleTheseCoords.to_numpy()
N = arr.shape[0]
print(N)
print(arr)

367
[['NotMorrison' 'Untitled Placemark' -124.3363757 40.26247792]
 ['NotMorrison' 'Untitled Placemark' -124.2375297 40.29336713]
 ['NotMorrison' 'Untitled Placemark' -124.0880253 40.33313018]
 ...
 ['NotMorrison' 'Untitled Placemark' -106.3437911 35.10539834]
 ['NotMorrison' 'Untitled Placemark' -106.3281701 35.11556836]
 ['NotMorrison' 'Untitled Placemark' -106.3113461 35.11224026]]


In [11]:
SampleTheseCoords

,Folder,Filename,Longitude,Latitude
0,NotMorrison,Untitled Placemark,-124.336376,40.262478
1,NotMorrison,Untitled Placemark,-124.237530,40.293367
2,NotMorrison,Untitled Placemark,-124.088025,40.333130
3,NotMorrison,Untitled Placemark,-123.972598,40.388938
4,NotMorrison,Untitled Placemark,-123.739353,40.463355
...,...,...,...,...
362,NotMorrison,Untitled Placemark,-106.356988,35.102934
363,NotMorrison,Untitled Placemark,-106.354295,35.204744
364,NotMorrison,Untitled Placemark,-106.343791,35.105398
365,NotMorrison,Untitled Placemark,-106.328170,35.115568


# Serial acquisition of images

In [ ]:
# from tqdm import tqdm

# start = time.time()
# print(start)
# for i, row in enumerate(tqdm(arr)):
#     folder = f'Downloads/{row[0]}/'
#     filename = f'{i:03}{row[1]}.png'
#     long = row[2]
#     lat = row[3]
#     dataset = RetrieveNAIPimage(lat, long)
#     image = dataset.first()
#     SaveNAIPimage(image ,lat, long, folder, filename, verbose = False)
# end = time.time()
# print(f'Elapsed: {(end-start)/60:5.1f} min')

# Dask Parallel Acquisition of images

In [12]:
import os
import dask

try:
    os.makedirs('data/geemap/Morrison/')
    os.makedirs('data/geemap/NotMorrison/')
    os.makedirs('data/geemap/train/NotMorrison')
    os.makedirs('data/geemap/val/NotMorrison')
    os.makedirs('data/geemap/train/Morrison')
    os.makedirs('data/geemap/val/Morrison')

except:
    print("folder already exist")

folder already exist


In [13]:
import os
import dask

try:
    os.makedirs('data/geemap/Morrison/')
    os.makedirs('data/geemap/NotMorrison/')
    os.makedirs('data/geemap/train/NotMorrison')
    os.makedirs('data/geemap/val/NotMorrison')
    os.makedirs('data/geemap/train/Morrison')
    os.makedirs('data/geemap/val/Morrison')

except:
    print("folder already exist")
    
lazy  = []
fileError = []
fileTried = []

def kernel(row):
    #print(row)
    folder = row[0]
    filename = row[1]
    long = float(row[2])
    lat = float(row[3])
    dataset = RetrieveNAIPimage(lat, long)
    image = dataset.first()
    SaveNAIPimage(image, lat, long, folder, filename, verbose = False)
    lazy.append(row)
    return row
    
ee.Initialize()
start = time.time()
print(start)
for i in range(0,N):
    row = arr[i]
    row[0] = f'data/geemap/{row[0]}/'
    row[1] = f'{i:03}{row[1]}'
    res = dask.delayed(kernel)(row)
    lazy.append(res)
start = time.time()    
dask.compute(*lazy)
end = time.time()
print(f'Elapsed: {(end-start)/60:5.1f} min')

folder already exist
1704822908.895526
Elapsed:   0.6 min


In [14]:
arr

array([['data/geemap/NotMorrison/', '000Untitled Placemark',
        -124.3363757, 40.26247792],
       ['data/geemap/NotMorrison/', '001Untitled Placemark',
        -124.2375297, 40.29336713],
       ['data/geemap/NotMorrison/', '002Untitled Placemark',
        -124.0880253, 40.33313018],
       ...,
       ['data/geemap/NotMorrison/', '364Untitled Placemark',
        -106.3437911, 35.10539834],
       ['data/geemap/NotMorrison/', '365Untitled Placemark',
        -106.3281701, 35.11556836],
       ['data/geemap/NotMorrison/', '366Untitled Placemark',
        -106.3113461, 35.11224026]], dtype=object)

In [15]:
Files = np.array(fileTried)
Files

array([['data/geemap/NotMorrison/', '011Untitled Placemark',
        '38.98105149', '-120.0259328', 'True'],
       ['data/geemap/NotMorrison/', '014Untitled Placemark',
        '41.04136102', '-112.5268842', 'True'],
       ['data/geemap/NotMorrison/', '041Untitled Placemark',
        '40.3547727', '-109.3494974', 'True'],
       ...,
       ['data/geemap/NotMorrison/', '172Untitled Placemark',
        '35.37423276', '-106.9917243', 'True'],
       ['data/geemap/Morrison/', '138pet4', '35.20343491',
        '-107.1277891', 'True'],
       ['data/geemap/NotMorrison/', '112Untitled Placemark',
        '35.39812434', '-107.1519635', 'True']], dtype='<U32')

In [16]:
FilesAfterAcquisition = pd.DataFrame(Files, columns=['Folder', 'Filename', 'Latitude', 'Longitude', 'Exists'])
FilesAfterAcquisition

,Folder,Filename,Latitude,Longitude,Exists
0,data/geemap/NotMorrison/,011Untitled Placemark,38.98105149,-120.0259328,True
1,data/geemap/NotMorrison/,014Untitled Placemark,41.04136102,-112.5268842,True
2,data/geemap/NotMorrison/,041Untitled Placemark,40.3547727,-109.3494974,True
3,data/geemap/NotMorrison/,050Untitled Placemark,40.42727535,-109.303103,True
4,data/geemap/NotMorrison/,048Untitled Placemark,40.42117962,-109.329868,True
...,...,...,...,...,...
362,data/geemap/Morrison/,122can1,35.18023895,-107.1361037,True
363,data/geemap/Morrison/,132pet8,35.20750464,-107.1308938,True
364,data/geemap/NotMorrison/,172Untitled Placemark,35.37423276,-106.9917243,True
365,data/geemap/Morrison/,138pet4,35.20343491,-107.1277891,True


In [17]:
FilesAfterAcquisition[ (FilesAfterAcquisition['Exists'] == 'True')].to_csv('FilesAfterAcquisition.csv')
FilesAfterAcquisition[ (FilesAfterAcquisition['Exists'] == 'False')].to_csv('FilesFailedAcquisition.csv')
FilesAfterAcquisition[ (FilesAfterAcquisition['Exists'] == 'True') & (FilesAfterAcquisition['Folder'] == 'data/geemap/NotMorrison/')  ]

,Folder,Filename,Latitude,Longitude,Exists
0,data/geemap/NotMorrison/,011Untitled Placemark,38.98105149,-120.0259328,True
1,data/geemap/NotMorrison/,014Untitled Placemark,41.04136102,-112.5268842,True
2,data/geemap/NotMorrison/,041Untitled Placemark,40.3547727,-109.3494974,True
3,data/geemap/NotMorrison/,050Untitled Placemark,40.42727535,-109.303103,True
4,data/geemap/NotMorrison/,048Untitled Placemark,40.42117962,-109.329868,True
...,...,...,...,...,...
353,data/geemap/NotMorrison/,107Untitled Placemark,35.38901124,-107.1541625,True
360,data/geemap/NotMorrison/,106Untitled Placemark,35.38644644,-107.1556753,True
361,data/geemap/NotMorrison/,102Untitled Placemark,35.41481825,-107.1622566,True
364,data/geemap/NotMorrison/,172Untitled Placemark,35.37423276,-106.9917243,True


In [18]:
FilesAfterAcquisition[ (FilesAfterAcquisition['Exists'] == 'True') & (FilesAfterAcquisition['Folder'] == 'data/geemap/Morrison/')  ]

,Folder,Filename,Latitude,Longitude,Exists
10,data/geemap/Morrison/,084nwnm03,36.66106573,-108.9484355,True
11,data/geemap/Morrison/,230bw10,35.5418331,-106.9145763,True
12,data/geemap/Morrison/,073ut03,37.01290363,-109.1441284,True
16,data/geemap/Morrison/,023utblan09,37.54037993,-109.5979999,True
20,data/geemap/Morrison/,144priet1,35.32156381,-107.1173113,True
...,...,...,...,...,...
358,data/geemap/Morrison/,240bw9,35.54232586,-106.9111407,True
359,data/geemap/Morrison/,208nety1,35.50870036,-106.9198833,True
362,data/geemap/Morrison/,122can1,35.18023895,-107.1361037,True
363,data/geemap/Morrison/,132pet8,35.20750464,-107.1308938,True


# Populate train & val folders

In [19]:
import glob
import os
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

### Use train test split to partition my list of file names

In [20]:
# Prep Morrison
df = pd.read_csv('FilesAfterAcquisition.csv')
MorrisonAll = df[df['Folder'] == 'data/geemap/Morrison/']['Filename'].tolist()

Morrison_train, Morrison_val = train_test_split(
    MorrisonAll,  test_size=0.10, random_state=42)

In [21]:
# Prep NotMorrison
NotMorrisonAll = df[df['Folder'] == 'data/geemap/NotMorrison/']['Filename'].tolist()

NotMorrison_train, NotMorrison_val = train_test_split(
    NotMorrisonAll,  test_size=0.10, random_state=42)

# Moved Train Test split images to associated folders

In [24]:
import shutil
import os

MT_path = 'data/geemap/train/Morrison/'
MV_path = 'data/geemap/val/Morrison/'
NT_path = 'data/geemap/train/NotMorrison/'
NV_path = 'data/geemap/val/NotMorrison/'

MSrc_path = 'data/geemap/Morrison/'
NSrc_path = 'data/geemap/NotMorrison/'

if not os.path.exists(MT_path):
       os.makedirs(MT_path)
if not os.path.exists(MV_path):
       os.makedirs(MV_path)
if not os.path.exists(NT_path):
       os.makedirs(NT_path)
if not os.path.exists(NV_path):
       os.makedirs(NV_path)
        
for fn in Morrison_train:
    fnm = fn+'.png'
    shutil.copyfile(f'{MSrc_path}{fnm}', f'{MT_path}{fnm}')
for fn in Morrison_val:
    fnm = fn+'.png'
    shutil.copyfile(f'{MSrc_path}{fnm}', f'{MV_path}{fnm}')
        
for fn in NotMorrison_train:
    fnm = fn+'.png'
    shutil.copyfile(f'{NSrc_path}{fnm}', f'{NT_path}{fnm}')
for fn in NotMorrison_val:
    fnm = fn+'.png'
    shutil.copyfile(f'{NSrc_path}{fnm}', f'{NV_path}{fnm}')


In [25]:
print("Done!")

Done!
